In [ ]:
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
import ipywidgets as ipw

from ipywidgets import interact
from random import choice
%matplotlib inline

In [ ]:
# Define a function to map a 1D array to a 2D/3D lattice, only works on lattice of equal-lateral size in Cartesian system
def map(index, size, dimension): 
    if dimension == 1:
        return index;
    if dimension == 2:
        x = index%size;
        y = np.floor(index/size);
       # return vp.vector(x,y,0);
        return [x, y];
    if dimension == 3:
        sub_index = index%(size**2);
        x = sub_index%size;
        y = np.floor(sub_index/size);
        z = np.floor(index/size**2);
       # return vp.vector(x,y,z)
        return [x, y, z];

In [ ]:
# Define a function to map the 3D vector back into the 1D array index
def antimap(array, size, dimension):
    if dimension == 1:
        return array[0];
    if dimension == 2:
        return array[0] + array[1]*size;
    if dimension == 3:
        return array[0] + array[1]*size + array[2]*size*size;

In [ ]:
# Define a function to find the index of the nearest neighbours
def nNbr(index, size, dimension):
    if dimension == 1:
        return [(index+size-1)%size, (index+size+1)%size]; # apply periodic condition
    
    if dimension == 2:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1]), size);
        south = np.mod(np.add(pos,[0,size-1]), size);
        east = np.mod(np.add(pos,[size+1,0]), size);
        west = np.mod(np.add(pos,[size-1,0]),size);
        return [antimap(north, size, 2), antimap(south, size, 2), antimap(east, size, 2), antimap(west, size, 2)];
    
    if dimension == 3:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1,0]), size);
        south = np.mod(np.add(pos,[0,size-1,0]), size);
        east = np.mod(np.add(pos,[size+1,0,0]), size);
        west = np.mod(np.add(pos,[size-1,0,0]),size);
        up = np.mod(np.add(pos,[0,0,size+1]), size);
        down = np.mod(np.add(pos,[0,0,size-1]), size);
        return [antimap(north, size, 3), antimap(south, size, 3), antimap(east, size, 3), antimap(west, size, 3), antimap(up, size, 3), antimap(down, size, 3)];

In [ ]:
# Set up the system parameters
temp = 15; # in [K] 
L = 50; # Define the lattice size
dims = 2; # Define the dimensionality of the lattice
N = L**dims; # Compute the total number of the sites
kB = 8.617E-5 # Boltzmann constant in [eV/K]
J = -10.0*kB; # Define the coupling strength between spins
beta = 1/(temp*kB);

In [ ]:
# Set up array containers for data manipulation and initial conditions
#lattice = np.empty((N*N,3),int); # Create a container for the lattice sites
lattice = np.empty((N,dims),float); # Create a container for site positions in the lattice
spinV = np.empty(N,int); # Create a container for the spins for all the lattice sites
spinC = np.chararray(N); # Create a container to store color values (spin visualization)
nIdx = np.empty((N,dims*2), np.intp); # Create a container to store the indices of the nearest neighbours of each site
probs = np.empty(N,np.float); # Create a container to store in each iteration spin-flip probability
for ii in range(N):
    lattice[ii] = np.subtract(map(ii,L,dims),L/2);
    spinV[ii] = 1; # Create a container to store the values of the spins
    spinC[ii] = 'r';
    nIdx[ii] = nNbr(ii, L, dims);
mag = [sum(spinV)]; # Initializat a variable for the total magnitization of the lattice

In [ ]:
# Optional: Randomization of the initial condition
for ii in range(int(N/2)): # Randomly choose half of the lattice to flip the spin 
    kk = choice(range(N));
    spinV[kk] = -1;
    spinC[kk] = 'k'; # color all the spin down as black
mag = [sum(spinV)]; # Calculate the current total magnetization

In [ ]:
# !!This step will take a while depending on the number of iterations and the speed of the computer!!
# Pre-define the total iteration number and pre-calculate each step to save time when plotting
MaxIteration = 200; # Start the iteration count
tt = range(MaxIteration+1); # Initiate a array for plotting the magnetization
spinC = np.expand_dims(spinC, axis = 0);
ruler = np.random.rand(MaxIteration,len(spinV)); # Initiate a matrix to store pre-calculated random numbers for all iterations
tempC = np.empty(N,str); # create an empty array to store colors (spins) at each iteration to update the lattice
for time in range(MaxIteration):
    tempSpin = spinV; # Create a temporary container to store updated spins
    for ii in range(len(spinV)):
        nIdx[ii] = nNbr(ii, L, dims);
        probs[ii] = np.exp(beta*J*sum(spinV[nIdx[ii]])*spinV[ii])/np.exp(-beta*J*sum(spinV[nIdx[ii]])*spinV[ii]);
        tempSpin[ii] = -spinV[ii] if probs[ii] > ruler[time,ii] else spinV[ii];
        tempC[ii] = 'r' if tempSpin[ii] > 0 else 'k';
    spinC = np.append(spinC, np.expand_dims(tempC,axis = 0), axis = 0); 
    spinV = tempSpin;
    mag.append(sum(tempSpin));

In [ ]:
# Define the function that updates the plot
def update(time):
    fig = plt.figure(figsize=(15,9)); # Initialize a figure
    figa = fig.add_subplot(121);
    plt.subplots_adjust(left=0, bottom=0.25)
    Ising = figa.scatter(lattice[:,0],lattice[:,1], s=90, c = spinC[time,:], marker = 's', edgecolor = None);
    figb = fig.add_subplot(122);
    figb.plot(tt, mag,'o-', c='k');
    figb.plot(tt[time], mag[time],'o',c='r');
    plt.subplots_adjust(left=0,bottom=0.25);
interact(update,time=ipw.IntSlider(min=0, max=MaxIteration, step=1, value=0));